<a href="https://colab.research.google.com/github/HunterWelsch/BigData-AWS-RDS/blob/master/Challenge_16_vg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

In [0]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-02-16 19:45:02--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.11’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.79MB/s    in 0.2s    

2020-02-16 19:45:02 (4.79 MB/s) - ‘postgresql-42.2.9.jar.11’ saved [914037/914037]



In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigDataHW").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [0]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
vg_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)

In [0]:
from pyspark.sql.types import IntegerType, DateType
vg_data_df = vg_data_df.withColumn("customer_id", vg_data_df["customer_id"].cast(IntegerType()))
vg_data_df = vg_data_df.withColumn("product_parent", vg_data_df["product_parent"].cast(IntegerType()))
vg_data_df = vg_data_df.withColumn("review_date", vg_data_df["review_date"].cast(DateType()))
vg_data_df = vg_data_df.withColumn("star_rating", vg_data_df["star_rating"].cast(IntegerType()))
vg_data_df = vg_data_df.withColumn("helpful_votes", vg_data_df["helpful_votes"].cast(IntegerType()))
vg_data_df = vg_data_df.withColumn("total_votes", vg_data_df["total_votes"].cast(IntegerType()))

In [0]:
customers = vg_data_df.groupby("customer_id").count().withColumnRenamed("count", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   26079415|             1|
|   12521601|             2|
|    4593696|             1|
|    1468259|             1|
|   38173350|             1|
|    3953141|             2|
|   20598424|             1|
|   46625306|             1|
|   47160828|             2|
|    1337034|             1|
|   14231016|             1|
|   49785748|             1|
|     712525|             1|
|   35339694|             1|
|    8211293|             1|
|   41832251|             1|
|    9037272|             1|
|   52036002|             1|
|   49514578|             1|
|   22875949|             1|
+-----------+--------------+
only showing top 20 rows



In [0]:
review_id_table = vg_data_df.select("review_id", "customer_id", "product_id", "product_parent", "review_date")

In [0]:
products_table = vg_data_df.select("product_id", "product_title")

In [0]:
vine_table = vg_data_df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine")

In [0]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cukxpdx4oe4k.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"postgres",
          "password": "Umiami#11",
          "driver":"org.postgresql.Driver"}

In [0]:
!pip install pyspark

In [0]:
# Write DataFrame to active_user table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [0]:
# Write DataFrame to active_user table in RDS
products_table.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [0]:
# Write DataFrame to active_user table in RDS
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)